In [ ]:
import json
import glob
import json
import os
import cv2
import numpy as np
from PIL import Image, ImageDraw
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils


<h1> Get base dataset from production database </h1>

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
query = """
    SELECT * FROM keypoint_annotations
    WHERE pen_id=5 
    AND keypoints -> 'leftCrop' is not null
    AND keypoints -> 'rightCrop' is not null
    LIMIT 10;
"""

df = rds_access_utils.extract_from_database(query)
df.head()

<h1> Generate full-res crops and labels around body part </h1>

In [ ]:
def enhance(image, clip_limit=5):
    # convert image to LAB color model
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # split the image into L, A, and B channels
    l_channel, a_channel, b_channel = cv2.split(image_lab)

    # apply CLAHE to lightness channel
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(8, 8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L channel with the original A and B channel
    merged_channels = cv2.merge((cl, a_channel, b_channel))

    # convert image from LAB color model back to RGB color model
    final_image = cv2.cvtColor(merged_channels, cv2.COLOR_LAB2BGR)
    return final_image


In [ ]:
body_part = 'UPPER_LIP'
box_width = 224
output_root = '/root/data/alok/biomass_estimation/playground/akpr'

for idx, row in df.iterrows():
    
    # download left image
    left_image_url = row.left_image_url
    left_image_f, _, _ = s3_access_utils.download_from_url(left_image_url)
    left_image = enhance(cv2.imread(left_image_f))
    
    
    # select random crop containing body part in question
    keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in row.keypoints['leftCrop']}
    body_part_coordinates = keypoints['UPPER_LIP']
    
    x0 = max(0, body_part_coordinates[0] - np.random.randint(box_width * 0.3, box_width * 0.7))
    x1 = min(x0 + box_width, left_image.shape[1])
    
    y0 = max(0, body_part_coordinates[1] - np.random.randint(box_width * 0.3, box_width * 0.7))
    y1 = min(y0 + box_width, left_image.shape[0])
    
    crop = left_image[y0:y1, x0:x1]
    
    # save crop to disk
    left_crop_f_name = os.path.basename(left_image_url)
    left_crop_f = os.path.join(output_root, 'images', left_crop_f_name)
    cv2.imwrite(left_crop_f, crop)
    
    # save label to disk
    label = [body_part_coordinates[0] - x0, body_part_coordinates[1] - y0]
    print(label)
    label_f_name = left_crop_f_name.replace('.jpg', '.json')
    label_f = os.path.join(output_root, 'labels', label_f_name)
    with open(label_f, 'w') as f:
        json.dump(label, f)
    
    
    
 

In [ ]:
image_fs = sorted(glob.glob(os.path.join(output_root, 'images', '*.jpg')))
label_fs = sorted(glob.glob(os.path.join(output_root, 'labels', '*.json')))

In [ ]:
idx = 9
im = Image.open(image_fs[idx])
label = json.load(open(label_fs[idx]))
draw = ImageDraw.Draw(im)
draw.ellipse((label[0]-2, label[1]-2, label[0]+2, label[1]+2), fill='red')
im